# Answering Business Question Using SQL

In this project we'll work with the Chinook database(chinook.db).
Here's a list of tables in the database:

* employee
* customer
* invoice
* invoice_line
* playlist
* playlist_track
* track
* media_type
* artist
* album
* genre

### Overview of the Data

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT 
    name, 
    type
FROM sqlite_master
WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * FROM customer
LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


### Selecting Albums to Purchase

In [4]:
%%sql
WITH 
    track_sold_usa AS
        (
            SELECT il.* FROM invoice_line il
            INNER JOIN invoice i ON i.invoice_id = il.invoice_id
            INNER JOIN customer c ON c.customer_id = i.customer_id
            WHERE c.country = 'USA'
        )
SELECT g.name Genre, COUNT(ts.invoice_line_id) track_sold, 
CAST(count(ts.invoice_line_id) AS FLOAT) / 
        (SELECT COUNT(*) 
        FROM track_sold_usa) percentage_sold
FROM track_sold_usa ts
INNER JOIN track t ON t.track_id = ts.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY track_sold DESC
LIMIT 10;

Done.


Genre,track_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Above is the top 10 list of the most sold genre of music in the USA. Now we are going to pick the top 3 genre, and find artists who have sold the most in the top 3 genres. 

#### Most sold artists or bands in rock, punk or metal

In [5]:
%%sql
WITH artists AS
    (
        SELECT DISTINCT ar.name artist_name, g.name genre,
            t.track_id track_id
        FROM artist ar
            INNER JOIN album al ON al.artist_id = ar.artist_id
            INNER JOIN track t ON t.album_id = al.album_id
            INNER JOIN genre g ON g.genre_id = t.genre_id
        WHERE (g.name = 'Rock') OR (g.name = 'Alternative & Punk' OR 
                             g.name = 'Metal')
    )
SELECT ar.artist_name name, ar.genre genre, 
    COUNT(il.invoice_line_id) track_sold FROM artists ar
INNER JOIN invoice_line il ON il.track_id = ar.track_id
GROUP BY 1
ORDER BY track_sold DESC
LIMIT 20;

Done.


name,genre,track_sold
Queen,Rock,192
Jimi Hendrix,Rock,187
Nirvana,Rock,130
Red Hot Chili Peppers,Rock,130
Pearl Jam,Rock,129
AC/DC,Rock,124
Guns N' Roses,Metal,124
Foo Fighters,Rock,121
The Rolling Stones,Rock,117
Metallica,Metal,106


It's no surprise that the top 6 most sold artists or bands are in the rock genre because counts for more than half of the music sold in the USA. We'll pick Queen, Gun'N'Roses and Green Day just to mix it up a little.  

### Analyzing Employee sales performance

Now we are going to find out how employees are performaing based on their total sales.

In [6]:
%%sql

WITH total_sales AS
    (
        SELECT c.customer_id customer_id, 
               c.support_rep_id support_rep_id,
               SUM(i.total) total
        FROM customer c
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        GROUP BY 1
    )
SELECT e.first_name || ' ' || e.last_name name, e.hire_date,
    SUM(ts.total) total_sales
FROM employee e
INNER JOIN total_sales ts ON ts.support_rep_id = e.employee_id
GROUP BY 1
ORDER BY 3 DESC;

Done.


name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


We notice that the difference in total sales is because the employees' hiring date. Jane has been the longest in the company and she comes first with the highest total sales, while Steve who was the most recent hire comes last.

### Analyzing sales by country

In [7]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )
SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );


Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.


### Albums vs Individual tracks

In [8]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.